In [1]:
import json 
import pandas as pd
import numpy as np
import datetime
import requests

In [2]:
# Opening JSON file 
f = open('neighbor-districts-modified.json') 
  
# returns JSON object as a dictionary 
data = json.load(f) 

# sorting districts alphabetically
districts_from_json = []
for i in data:
    districts_from_json.append(i)
districts_from_json = np.array(districts_from_json)
districts_from_json.sort()

# ids for districts
ids = np.arange(101,101+len(districts_from_json))

# Dictionary for storing ids of districts
Dict = {} 
for i in range(len(ids)):
    Dict[districts_from_json[i]]=ids[i]

# storing vertex 1 and vertex 2 of each edge
edge1 = []
edge2 = []
for i in range(len(districts_from_json)):
    for j in range(len(data[districts_from_json[i]])):
        edge1.append(Dict[districts_from_json[i]])
        edge2.append(Dict[data[districts_from_json[i]][j]])

# Creating dataframe
edge_list = {'District 1': edge1, 'District 2': edge2}
edge_list = pd.DataFrame(edge_list)

In [3]:
district_names_from_json = []
district_ids_from_json = []
for i in range(len(districts_from_json)):
    district_names_from_json.append(districts_from_json[i].split('/')[0])
    district_ids_from_json.append(districts_from_json[i].split('/')[1])

In [4]:
districts_to_complete_ids = {}
for i in range(len(district_names_from_json)):
    districts_to_complete_ids[district_names_from_json[i]] = district_names_from_json[i] + '/' + district_ids_from_json[i]

In [5]:
time_id_week = {}
time_id_month = {}
time_id_overall = {}
date = datetime.date(2020,3,15)
day = 1
while True:
    time_id_week[str(date)] = int(np.ceil(day/7))
    time_id_month[str(date)] = int(str(date).split('-')[1])-2
    time_id_overall[str(date)] = 1
    if date == datetime.date(2020,9,5):
        break
    day+=1
    date += datetime.timedelta(days=1)

date = datetime.date(2020,3,15)

In [6]:
r = requests.get('https://api.covid19india.org/v4/data-all.json')
dat = json.loads(r.text)

date = datetime.date(2020,3,15)

Date = []
State = []
District = []
Dist_state = []
numcases = []
weekid = []
monthid = []
overallid = []
pop = []

while True:
    for state in dat[str(date)]:
        flag_districts=0
        for key in dat[str(date)][state]:
            if key == 'districts':
                flag_districts=1
                for district in dat[str(date)][state]['districts']:
                    flag_delta=0
                    for key_delta in dat[str(date)][state]['districts'][district]:
                        if key_delta == 'delta':
                            flag_delta=1
                            flag_confimed=0
                            for key_confirmed in dat[str(date)][state]['districts'][district]['delta']:
                                if key_confirmed == 'confirmed':
                                    flag_confirmed=1
                                    Date.append(str(date))
                                    State.append(state)
                                    District.append(district)
                                    Dist_state.append(district+':'+state)
                                    numcases.append(dat[str(date)][state]['districts'][district]['delta']['confirmed'])
                                    weekid.append(time_id_week[str(date)])
                                    monthid.append(time_id_month[str(date)])
                                    overallid.append(time_id_overall[str(date)])
                                    if 'meta' in dat[str(date)][state]['districts'][district].keys():
                                        if 'population' in dat[str(date)][state]['districts'][district]['meta'].keys():
                                            pop.append(dat[str(date)][state]['districts'][district]['meta']['population'])
                                        else:
                                            pop.append(0)
                                    else:
                                        pop.append(0)
        if flag_districts == 0:
            flag_delta=0
            if state== 'TT':
                continue
            for key in dat[str(date)][state]:
                if key == 'delta':
                    falg_delta=1
                    flag_confirmed = 0
                    for key_confirmed in dat[str(date)][state]['delta']:
                        if key_confirmed == 'confirmed':
                            flag_confirmed = 1
                            Date.append(str(date))
                            State.append(state)
                            District.append('All Districts')
                            numcases.append(dat[str(date)][state]['delta']['confirmed'])
                            weekid.append(time_id_week[str(date)])
                            monthid.append(time_id_month[str(date)])
                            overallid.append(time_id_overall[str(date)])
                            Dist_state.append('All Districts:'+state)
                            pop.append(0)
        
    if date == datetime.date(2020,9,5):
        break
    date += datetime.timedelta(days=1)

District_wise_data = {'Date': Date,'State': State, 'District': District, 'District_state': Dist_state, 'cases': numcases, 'weekid': weekid, 'monthid': monthid, 'overallid': overallid, 'Population': pop}
District_wise_data = pd.DataFrame(District_wise_data)
District_wise_data.sort_values("Population", axis = 0, ascending = False, kind='mergesort', inplace = True, na_position ='last')
District_wise_data.sort_values("State", axis = 0, ascending = True, kind='mergesort', inplace = True, na_position ='last')
District_wise_data = District_wise_data.reset_index(drop=True)
# District_wise_data.to_csv('District_wise_data.csv')


In [7]:
populous_district = {}
i=0
while i<len(District_wise_data):
    curr = District_wise_data['State'][i]
    populous_district[District_wise_data['State'][i]] = District_wise_data['District'][i]
    i+=1
    while i<len(District_wise_data) and District_wise_data['State'][i] ==curr:
        i+=1

In [8]:
r = requests.get('https://api.covid19india.org/v4/data-all.json')
dat = json.loads(r.text)

date = datetime.date(2020,3,15)

Date = []
State = []
District = []
Dist_state = []
numcases = []
weekid = []
monthid = []
overallid = []
pop = []

while True:
    for state in dat[str(date)]:
        flag_districts=0
        for key in dat[str(date)][state]:
            if key == 'districts':
                flag_districts=1
                for district in dat[str(date)][state]['districts']:
                    flag_delta=0
                    for key_delta in dat[str(date)][state]['districts'][district]:
                        if key_delta == 'delta':
                            flag_delta=1
                            flag_confimed=0
                            for key_confirmed in dat[str(date)][state]['districts'][district]['delta']:
                                if key_confirmed == 'confirmed':
                                    flag_confirmed=1 
                                    Date.append(str(date))
                                    State.append(state)
                                    District.append(district)
                                    Dist_state.append(district+':'+state)
                                    numcases.append(dat[str(date)][state]['districts'][district]['delta']['confirmed'])
                                    weekid.append(time_id_week[str(date)])
                                    monthid.append(time_id_month[str(date)])
                                    overallid.append(time_id_overall[str(date)])
#                                     Dist_id.append(Dict[districts_to_complete_ids[district.lower()]])
                                    if 'meta' in dat[str(date)][state]['districts'][district].keys():
                                        if 'population' in dat[str(date)][state]['districts'][district]['meta'].keys():
                                            pop.append(dat[str(date)][state]['districts'][district]['meta']['population'])
                                        else:
                                            pop.append(0)
                                    else:
                                        pop.append(0)
        if flag_districts == 0:
            flag_delta=0
            if state== 'TT':
                continue
            for key in dat[str(date)][state]:
                if key == 'delta':
                    falg_delta=1
                    flag_confirmed = 0
                    for key_confirmed in dat[str(date)][state]['delta']:
                        if key_confirmed == 'confirmed':
                            flag_confirmed = 1
                            Date.append(str(date))
                            State.append(state)
                            District.append(populous_district[state])
                            numcases.append(dat[str(date)][state]['delta']['confirmed'])
                            weekid.append(time_id_week[str(date)])
                            monthid.append(time_id_month[str(date)])
                            overallid.append(time_id_overall[str(date)])
                            Dist_state.append(populous_district[state]+':'+state)
#                             Dist_id.append(Dict[districts_to_complete_ids[populous_district[state].lower()]])
                            pop.append(0)
        
    if date == datetime.date(2020,9,5):
        break
    date += datetime.timedelta(days=1)

District_wise_data = {'Date': Date,'State': State, 'District': District, 'District_state': Dist_state, 'cases': numcases, 'weekid': weekid, 'monthid': monthid, 'overallid': overallid, 'Population': pop, 'ID': np.zeros(len(numcases))}
District_wise_data = pd.DataFrame(District_wise_data)
# District_wise_data.to_csv('District_wise_data.csv')


In [9]:
for i in range(len(District_wise_data)):
    if District_wise_data['District'][i].lower()=='hamirpur' and District_wise_data['State'][i] == 'HP':
        District_wise_data['ID'][i] = Dict['hamirpur/316']
    elif District_wise_data['District'][i].lower()=='bilaspur' and District_wise_data['State'][i] == 'CT':
        District_wise_data['ID'][i] = Dict['bilaspur/190']
    elif District_wise_data['District'][i].lower()=='pratapgarh' and District_wise_data['State'][i] == 'UP':
        District_wise_data['ID'][i] = Dict['pratapgarh/546']
    elif District_wise_data['District'][i].lower()=='balrampur' and District_wise_data['State'][i] == 'CT':
        District_wise_data['ID'][i] = Dict['balrampur/148']
    elif District_wise_data['District'][i].lower()=='aurangabad' and District_wise_data['State'][i] == 'BR':
        District_wise_data['ID'][i] = Dict['aurangabad/133']
    elif District_wise_data['District'][i].lower() in districts_to_complete_ids.keys():
        District_wise_data['ID'][i] = Dict[districts_to_complete_ids[District_wise_data['District'][i].lower()]]
    else:
        District_wise_data['ID'][i] = 0

indexNames = District_wise_data[District_wise_data['ID'] == 0].index
District_wise_data.drop(indexNames , inplace=True)
District_wise_data = District_wise_data.reset_index(drop=True)

# District_wise_data.to_csv('District_wise_data.csv')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
D:\Anaconda3\lib\si

In [10]:
# week
week_data = {'districtid': District_wise_data['ID'], 'weekid': District_wise_data['weekid'], 'cases':District_wise_data['cases'] }
week_data = pd.DataFrame(week_data)

weeks = week_data['weekid'].max()
for i in range(1,weeks+1):
    Data1 = {'districtid': np.arange(101,101+len(districts_from_json)),'weekid': i*np.ones(len(districts_from_json)),'cases':np.zeros(len(districts_from_json)) }
    Data1 = pd.DataFrame(Data1)
    week_data = week_data.append(Data1)

sorted1 = week_data.sort_values("districtid", axis = 0, ascending = True, kind='mergesort', inplace = False, na_position ='last')
sorted2 = sorted1.sort_values("weekid", axis = 0, ascending = True, kind='mergesort', inplace = False, na_position ='last')
sorted2['dropped'] = 0
sorted2 = sorted2.reset_index(drop=True)

for i in range(len(sorted2)-1):
    if sorted2['districtid'][i] == sorted2['districtid'][i+1]:
        sorted2['cases'][i+1] += sorted2['cases'][i]
        sorted2['dropped'][i] = 1
indexNames = sorted2[ sorted2['dropped'] == 1].index
sorted2.drop(indexNames , inplace=True)
sorted2 = sorted2.drop(['dropped'], axis = 1)
sorted2 = sorted2.reset_index(drop=True)

sorted2.to_csv('cases-week.csv')
# sorted2.to_csv('cases-week.csv')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [11]:
# month
month_data = {'districtid': District_wise_data['ID'], 'monthid': District_wise_data['monthid'], 'cases':District_wise_data['cases'] }
month_data = pd.DataFrame(month_data)

months = month_data['monthid'].max()
for i in range(1,months+1):
    Data1 = {'districtid': np.arange(101,101+len(districts_from_json)),'monthid': i*np.ones(len(districts_from_json)),'cases':np.zeros(len(districts_from_json)) }
    Data1 = pd.DataFrame(Data1)
    month_data = month_data.append(Data1)

sorted1 = month_data.sort_values("districtid", axis = 0, ascending = True, kind='mergesort', inplace = False, na_position ='last')
sorted2 = sorted1.sort_values("monthid", axis = 0, ascending = True, kind='mergesort', inplace = False, na_position ='last')
sorted2['dropped'] = 0
sorted2 = sorted2.reset_index(drop=True)
for i in range(len(sorted2)-1):
    if sorted2['districtid'][i] == sorted2['districtid'][i+1]:
        sorted2['cases'][i+1] += sorted2['cases'][i]
        sorted2['dropped'][i] = 1
indexNames = sorted2[ sorted2['dropped'] == 1].index
sorted2.drop(indexNames , inplace=True)
sorted2 = sorted2.drop(['dropped'], axis = 1)
sorted2 = sorted2.reset_index(drop=True)
sorted2.to_csv('cases-month.csv') 

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [12]:
# overall
overall_data = {'districtid': District_wise_data['ID'], 'overallid': District_wise_data['overallid'], 'cases':District_wise_data['cases'] }
overall_data = pd.DataFrame(overall_data)

sorted1 = overall_data.sort_values("districtid", axis = 0, ascending = True, kind='mergesort', inplace = False, na_position ='last')
sorted2 = sorted1.sort_values("overallid", axis = 0, ascending = True, kind='mergesort', inplace = False, na_position ='last')
sorted2['dropped'] = 0
sorted2 = sorted2.reset_index(drop=True)
for i in range(len(sorted2)-1):
    if sorted2['districtid'][i] == sorted2['districtid'][i+1]:
        sorted2['cases'][i+1] += sorted2['cases'][i]
        sorted2['dropped'][i] = 1
indexNames = sorted2[ sorted2['dropped'] == 1].index
sorted2.drop(indexNames , inplace=True)
sorted2 = sorted2.drop(['dropped'], axis = 1)
sorted2 = sorted2.reset_index(drop=True)
sorted2.to_csv('cases-overall.csv') 

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
